In [1]:
import pandas as pd
import numpy as np
labels = pd.read_csv("/Users/mingliangang/Desktop/cifar10/trainLabels.csv")
labels.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [2]:
label2id = {x:i for i,x in enumerate(list(set(labels["label"])))}
id2label = {i:x for i,x in enumerate(list(set(labels["label"])))}
labelcon = lambda x: label2id[x]
labels['label'] = labels['label'].map(labelcon)
labels_ = labels['label'].as_matrix()

In [3]:
labels['label'].head()

0    3
1    1
2    1
3    9
4    2
Name: label, dtype: int64

In [4]:
labels_

array([3, 1, 1, ..., 1, 2, 2])

# Sanity check

In [5]:
labels['label'].map(lambda x: id2label[x]).head()

0          frog
1         truck
2         truck
3          deer
4    automobile
Name: label, dtype: object

# Tensorflow input pipeline

In [6]:
import os
from sklearn.model_selection import train_test_split
import cv2
train = np.array([cv2.imread("/Users/mingliangang/Desktop/cifar10/train/"+i) for i in os.listdir("/Users/mingliangang/Desktop/cifar10/train")]).astype(np.float32)
X_train, X_val, y_train, y_val = train_test_split(train, labels_, test_size=0.33, random_state=42)

In [7]:
X_train.shape

(33500, 32, 32, 3)

# Resnet Model (CIFAR 10)

In [8]:
from models import resnet
import tensorflow as tf

batch_size = 128
GPU = True
summaries_dir = "/Users/mingliangang/Desktop/resnet2/"

if GPU:
    with tf.device('/device:GPU:0'):
        X = tf.placeholder("float", [batch_size, 32, 32, 3])
        Y = tf.placeholder("float", [batch_size, 10])
        label = tf.placeholder("int64",[batch_size])
        learning_rate = tf.placeholder("float", [])
        
        global_step_tensor = tf.Variable(0, trainable=False, name='global_step')
        
        net = resnet(X,20)
        
        cross_entropy = -tf.reduce_sum(Y*tf.log(net))
        tf.summary.scalar('cross_entropy', cross_entropy)
        correct_prediction = tf.equal(label,tf.argmax(net,axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
        opt = tf.train.MomentumOptimizer(learning_rate, 0.9)
        train_op = opt.minimize(cross_entropy,global_step=global_step_tensor)
        
    merged = tf.summary.merge_all()
    config = tf.ConfigProto(allow_soft_placement = True)
    sess = tf.Session(config = config)
    writer = tf.summary.FileWriter(summaries_dir + '/gpu', sess.graph)
    sess.run(tf.global_variables_initializer())
    
else:
    with tf.device('/device:CPU:0'):
        X = tf.placeholder("float", [batch_size, 32, 32, 3])
        Y = tf.placeholder("float", [batch_size, 10])
        label = tf.placeholder("int64",[batch_size])
        learning_rate = tf.placeholder("float", [])
        
        global_step = tf.Variable(0, trainable=False, name='global_step')
        
        net = resnet(X,20)
        
        cross_entropy = -tf.reduce_sum(Y*tf.log(net))
        tf.summary.scalar('cross_entropy', cross_entropy)
        correct_prediction = tf.equal(label,tf.argmax(net,axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
        opt = tf.train.MomentumOptimizer(learning_rate, 0.9)
        train_op = opt.minimize(cross_entropy)
        
    merged = tf.summary.merge_all()
    sess = tf.Session()
    writer = tf.summary.FileWriter(summaries_dir + '/cpu', sess.graph)
    sess.run(tf.global_variables_initializer())

Tensor("Shape:0", shape=(4,), dtype=int32, device=/device:GPU:0)


In [9]:
#Multiple epochs
from tqdm import tqdm
n_epochs = 1
for _ in tqdm(range(n_epochs)):
  while True:
    try:
      randix = np.random.randint(len(X_train),size=batch_size)
      x_batch = X_train[randix]
      y_batch = y_train[randix]
      summary,loss,acc,_ = sess.run([merged,cross_entropy,accuracy,train_op], feed_dict={label: y_batch,X: x_batch, Y: np.eye(10)[y_batch],learning_rate: 0.001})
      steps = tf.train.global_step(sess, global_step_tensor)
      if steps % 100 == 0:
          print("loss : {} accuracy : {}".format(loss,acc))
          writer.add_summary(summary, steps)
    except ValueError:
      print(loss)
      break

  0%|          | 0/1 [00:00<?, ?it/s]

loss : 296.8796081542969 accuracy : 0.140625
loss : 293.93682861328125 accuracy : 0.125
loss : 294.0349426269531 accuracy : 0.140625
loss : 295.1753845214844 accuracy : 0.1015625
loss : 295.8197326660156 accuracy : 0.0390625
loss : 294.703369140625 accuracy : 0.078125
loss : 294.9972839355469 accuracy : 0.0859375
loss : 295.668701171875 accuracy : 0.078125
loss : 295.05029296875 accuracy : 0.0703125
loss : 294.7466125488281 accuracy : 0.125
loss : 293.6047668457031 accuracy : 0.1484375
loss : 294.7416076660156 accuracy : 0.0859375
loss : 294.7076416015625 accuracy : 0.109375
loss : 294.81072998046875 accuracy : 0.125
loss : 295.4272155761719 accuracy : 0.09375
loss : 294.8186340332031 accuracy : 0.1484375
loss : 295.0841064453125 accuracy : 0.1015625
loss : 294.94305419921875 accuracy : 0.1015625
loss : 294.7513122558594 accuracy : 0.109375
loss : 294.7073974609375 accuracy : 0.09375
loss : 294.7951965332031 accuracy : 0.1328125
loss : 295.5307922363281 accuracy : 0.0859375
loss : 295.

KeyboardInterrupt: 

From what i read online, it seems that this implementation of resnet is not very good. 